### Step 1 of 2: Create a Crown Height Model (CHM)
uses a LiDAR input point cloud to develop a crown height model (CHM) that can be used for individual tree segmentation.
#### Input Arguments:
<small>
<b><u>project_basename : string  </b></u>  <br>
this is used to name output files and layers  <br>  
    
    
<b><u>input_las_file : .las dataset</b></u>  
file path to the .las dataset that the crown height model will be built from  

    
<b><u>input_mask_file : polygon</b></u>  
file path to a polygon feature class that will be used as the boundary mask for the analysis  

<b><u>chm_result_directory : folder path</b></u>  
file path to the folder where the outputs will be saved

<b><u>output_wkid : integer</b></u>  
the Well Known ID used to identify a predefined coordinate reference system.  
see https://github.com/Esri/projection-engine-db-doc/tree/main/json for a full list  
for flagstaff, use: <b>wkid 26949 = NAD 1983 StatePlane Arizona Central FIPS 0202 (Meters)</b>  
<i>the input coordinate system must be in meters</i>

<b><u>resolution in meters : float (meters)</b></u>  
the resolution of the output rasters, in meters.
processes in tree segementation will depend on this value, because window size in segmentation analysis is set in pixels.
</small>

In [1]:
import os
from pycrown4arcgis import ProcessCHM

# project naming, input features, and output directory:
project_basename = "SterlingPoint"
input_las_file = os.path.join("D:", os.sep, "TreeSegmentation", "SterlingPointInputs", "SterlingPointLidar.las")
input_mask_file = os.path.join("D:", os.sep, "TreeSegmentation", "TreeSegmentation.gdb", "SterlingPointAnalysisMask")
chm_result_directory = os.path.join("D:", os.sep, "TreeSegmentation", "SterlingPointResultChm")

# set the analysis parameters here:
output_wkid = 26949
output_resolution = 0.5
pro_project = arcpy.mp.ArcGISProject("CURRENT")

# call the method using the above parameters:
ProcessCHM(
    project_basename = project_basename,
    input_las_file = input_las_file,
    input_mask_file = input_mask_file,
    chm_result_directory = chm_result_directory,
    output_wkid = output_wkid,
    output_resolution = output_resolution,
    pro_project = arcpy.mp.ArcGISProject("CURRENT")
)



--> Prep map by removing existing CHM group layer to remove locks...
<-- Map prepped.

--> Filter LAS dataset to exclude existing buildings...
<-- Filtered.

--> Project LAS dataset to spatial reference in meters...
<-- Projected.

--> Clean up intermediate LAS data...
<-- Cleaned up.

--> Create DSM from LAS dataset...
<-- DSM created.

--> Create DTM from LAS dataset using only bare ground classified points...
<-- DTM created.

--> Create CHM by subtracting DTM from DSM...
<-- CHM created.

--> Add new layers to map...
<-- Layers added.

- ProcessCHM Done.


### Step 2 of 2: Individual Tree Segmentation
an updated implementation optimized for ArcGIS Pro, based on the PyCrown package developed by Dr Jan Schindler:
https://github.com/manaakiwhenua/pycrown
#### Input Arguments:
<small>
<b><u>project_basename : string</b></u>  
this is used to name output files and layers  

<b><u>pycrown_result_directory : folder path</b></u>  
file path to the folder where the outputs will be saved

<b><u>chm_result_directory : folder path</b></u>  
this is the result directory from above, used to locate the previous outputs  

<b><u>outputs from ProcessCHM : .las and .tif files</b></u>  
paths to the outputs from above:  
- pycrown_LAS  
- pycrown_CHM  
- pycrown_DSM  
- pycrown_DTM  

<b><u>input_mask_file : polygon</b></u>  
file path to a polygon feature class that will be used as the boundary mask for the analysis

<b><u>chm_smooth_window_size : integer (pixels)</b></u>  
default : 1  
the window size of the smoothing process, in CHM pixels  

<b><u>chm_smooth_circular : boolean</b></u>  
Default : True

<b><u>tree_detection_window_size : float (pixels)</b></u>  
Default : 1.5  
window size in CHM pixels  

<b><u>tree_detection_min_height : float (meters)</b></u>  
Default : 1.4  
height where tree has a DBH value and is considered a tree  

<b><u>crown_delineation_algorithm : string</b></u>  
Default : watershed_skimage  
choose one of the following algorithms:  
- dalponte_numba  
- dalponteCIRC_numba  
- watershed_skimage  

<b><u>crown_delineation_th_seed : float (between 0 and 1)</b></u>  
Default : 0.45 (per pycrown4Boreal)  
Growing threshold 1:  
A pixel is added to a region if its height is greater than the tree height multiplied by this value. It should be between 0 and 1.  

<b><u>crown_delineation_th_crown : float (between 0 and 1)</b></u>  
Default : 0.55 (per pycrown4Boreal)  
Growing threshold 2:  
A pixel is added to a region if its height is greater than the current mean height of the region multiplied by this value. It should be between 0 and 1.  

<b><u>crown_delineation_th_tree : float (meters)</b></u>  
Default : 2 (per pycrown4Boreal)  
Threshold below which a pixel cannot be a tree.  

<b><u>crown_delineation_max_crown : float (meters)</b></u>  
Default : 10 (per pycrown4Boreal)  
Maximum value of the crown diameter of a detected tree (in pixels)  
</small>

In [1]:
import os
from pycrown4arcgis import ProcessCrowns

# project naming and directories:
project_basename = "SterlingPoint"
pycrown_result_directory = os.path.join("D:", os.sep, "TreeSegmentation", "SterlingPointResultPyCrown")

# outputs from ProcessCHM:
chm_result_directory = os.path.join("D:", os.sep, "TreeSegmentation", "SterlingPointResultChm")
pycrown_LAS = os.path.join(chm_result_directory, f"{project_basename}_Projected.las")
pycrown_CHM = os.path.join(chm_result_directory, f"{project_basename}_CHM.tif")
pycrown_DSM = os.path.join(chm_result_directory, f"{project_basename}_DSM.tif")
pycrown_DTM = os.path.join(chm_result_directory, f"{project_basename}_DTM.tif")
input_mask_file = os.path.join("D:", os.sep, "TreeSegmentation", "TreeSegmentation.gdb", "SterlingPointAnalysisMask")

# set the analysis parameters here:
chm_smooth_window_size = 1
chm_smooth_circular = True 
tree_detection_window_size = 1.5
tree_detection_min_height = 1.4
crown_delineation_algorithm = "dalponteCIRC_numba"
crown_delineation_th_seed = 0.45
crown_delineation_th_crown = 0.55
crown_delineation_th_tree = 1.4
crown_delineation_max_crown = 10

# call the method using the above parameters:
ProcessCrowns(
    project_basename = project_basename,
    pycrown_result_directory = pycrown_result_directory,
    pycrown_CHM = pycrown_CHM,
    pycrown_DTM = pycrown_DTM,
    pycrown_DSM = pycrown_DSM,
    pycrown_LAS = pycrown_LAS,
    input_mask_file = input_mask_file,
    chm_smooth_window_size = chm_smooth_window_size,
    chm_smooth_circular = chm_smooth_circular,
    tree_detection_window_size = tree_detection_window_size,
    tree_detection_min_height = tree_detection_min_height,
    crown_delineation_algorithm = crown_delineation_algorithm,
    crown_delineation_th_seed = crown_delineation_th_seed,
    crown_delineation_th_crown = crown_delineation_th_crown,
    crown_delineation_th_tree = crown_delineation_th_tree,
    crown_delineation_max_crown = crown_delineation_max_crown,
    pro_project = arcpy.mp.ArcGISProject("CURRENT")
)



--> Prep map by removing existing PyCrown group layer to remove locks...
<-- Map prepped.

CREATING PYCROWN OBJECT:
--> Loading input files...
    - Reading .las file:
        - Filtering las classification 2 (ground) and 6 (building):
        - Filtered.
        - Loading las into DataFrame:
        - las points loaded.
<-- Input Files Loaded.
<-- Set up PyCrown trees object...
<-- Created.

FILTER CHM TO SMOOTH AND REMOVE OUTLIERS:
--> Applying a median filter (scipy.ndimage.filters.median_filter)...
<-- CHM Filtered.

DETECTING INDIVIDUAL TREES.
--> Apply scipy.ndimage.filters.maximum_filter to find maximum value within window size...
<-- Filter applied.
--> When multiple pixels in the window have the same maxima,
    apply scipy.ndimage.center_of_mass to find the weighted average...
<-- Applied; Tree Detection Complete:
    - Number of trees detected: {len(self.trees)}

DELINEATE CROWNS:
--> Delineating crowns using the dalponteCIRC_numba algorithm...
<-- Delineation took 0.216s:
